# 환경셋팅
Qlora를 위한 환경셋팅  
클라우드에서 한다면 터미널에서 huggingface-cli login 으로 토큰 작성  
이 버전 아니면 안됨!!! (무조건 오류남.)

In [1]:
!pip install transformers==4.51.3 peft==0.14.0 trl==0.15.2

# (4) 양자화 및 데이터셋 처리 라이브러리 설치
!pip install bitsandbytes==0.45.3 datasets==3.3.2 accelerate==1.4.0 evaluate==0.4.3

# (5) 기타 필요한 패키지 설치
!pip install sentencepiece protobuf tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 49.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 79.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

# 1차 학습코드

1차 학습코드. train test split -> 후에 Jsonl Map 해서 학습시키기

In [2]:
from datasets import load_dataset

# JSONL 파일을 Dataset으로 불러오기
dataset = load_dataset("json", data_files="blog_sft_data.jsonl", split="train")

# 'prompt'와 'completion'을 대화 형식의 messages로 변환하는 함수 정의
def to_conversation(sample):
    return {
        "messages": [
            {"role": "user", "content": sample["prompt"]},       # 사용자 프롬프트
            {"role": "assistant", "content": sample["completion"]}  # 모델이 생성해야 할 답변
        ]
    }

# 데이터셋의 각 샘플을 변환하고, 필요 없는 원본 열(drop) 제거
dataset = dataset.map(to_conversation, remove_columns=dataset.column_names)
#  dataset.train_test_split(test_size=0.1)
train_data = data_split["train"]
eval_data  = data_split["test"]


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForImageTextToText, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

# 1) 모델과 토크나이저 로드 (4-bit 양자화 설정)
model_id = "google/gemma-3-4b-it"  # 파인튜닝할 Gemma 3 4B Instruction Tuned 모델


# torch랑 bfloat16 설정 (최신 글카면 bfloat16 사용 가능합니다)
if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

# BitsAndBytes 4-bit 양자화 설정 생성:contentReference[oaicite:16]{index=16}
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,      # 2중 양자화 사용 (메모리 효율 향상)
    bnb_4bit_quant_type="nf4",           # NF4 양자화 타입
    bnb_4bit_compute_dtype=torch_dtype   # bfloat 16 사용
)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=quant_config,
    device_map="auto",
    attn_implementation="eager",   # FlashAttention 미사용 => 이상하게 버그발생
    torch_dtype=torch_dtype
)

# 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2) LoRA 어댑터 구성 설정:contentReference[oaicite:18]{index=18}
peft_config = LoraConfig(
    r=16,              # Rank 
    lora_alpha=16,     # LoRA scaling
    lora_dropout=0.05, # 드롭아웃 비율 (QLoRA 논문 권장 0.05)
    bias="none",       # Bias는
    target_modules="all-linear",  # 모든 Linear 계층에 LoRA 적용
    task_type="CAUSAL_LM"         # 언어모델 과제 타입
)
# (참고) Gemma 모델의 경우 embedding과 LM 헤드 등 일부 모듈은 LoRA 적용 대상에서 제외하거나 별도 저장 필요

# 3) 파인튜닝 하이퍼파라미터 설정 (SFTConfig 사용)
train_args = SFTConfig(
    output_dir="gemma3-4b-blog-qlora",   # 출력 디렉토리 (및 HF Hub 리포지토리 이름)
    max_seq_length=512,                 # 최대 시퀀스 길이 (512 토큰)
    packing=False,                      # (여기서는 packing 미사용; 필요시 True로 설정 가능)
    num_train_epochs=3,                 # 에포크 수 (예: 3)
    per_device_train_batch_size=1,      # GPU 한 장당 배치 사이즈
    gradient_accumulation_steps=4,      # 그래디언트 누적 스텝 
    gradient_checkpointing=True,        # 그래디언트 체크포인팅 활성화 (메모리 절약):
    optim="adamw_torch_fused",          # PyTorch fused AdamW 옵티마이저 사용:
    learning_rate=2e-4,                 # 학습률 
    max_grad_norm=0.3,                  # 그래디언트 클리핑 임계값 
    warmup_ratio=0.03,                  # 워밍업 비율
    lr_scheduler_type="constant",       # ???
    fp16=True if torch_dtype == torch.float16 else False,   
    bf16=True if torch_dtype == torch.bfloat16 else False,  
    logging_steps=10,                   # 로깅 간격 
    save_strategy="epoch",              # 체크포인트 저장 간격 (에포크마다 저장)
    push_to_hub=False,                 
    report_to="tensorboard"            

)

# 4) SFTTrainer 초기화 및 훈련 시작
trainer = SFTTrainer(
    model=model,
    args=train_args,
    train_dataset=train_data,      # 앞서 준비한 학습 Dataset
    eval_dataset=eval_data,        # (옵션) 검증 Dataset
    peft_config=peft_config,
    # 데이터 처리용 토크나이저/전처리 클래스 지정 
    tokenizer=tokenizer  
)

# 모델 훈련 실행 (학습 진행 상황은 로그로 출력됨)
trainer.train()

# 5) 학습 완료 후 어댑터 모델 저장
trainer.save_model()  # LoRA 어댑터 가중치를 output_dir에 저장


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,13.493100
20,9.349400
30,8.393000
40,8.051400
50,8.011200
60,7.518700
70,7.194700
80,7.074200
90,6.966900
100,7.170700


/venv/main/lib/python3.12/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/venv/main/lib/python3.12/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/venv/main/lib/python3.12/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/venv/main/lib/python3.12/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

# Qlora 추가학습 코드
1차 실험에서 이어나가는 코드 (Json 다른거 이름만 같게하면 무조건 추가학습되니 한번 짜고 계속 써먹을것)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

# === 0. Dataset 불러오기 및 전처리 ===
dataset = load_dataset("json", data_files="blog_sft_data.jsonl", split="train")

def to_conversation(sample):
    return {
        "messages": [
            {"role": "user", "content": sample["prompt"]},
            {"role": "assistant", "content": sample["completion"]}
        ]
    }

dataset = dataset.map(to_conversation, remove_columns=dataset.column_names)
data_split = dataset.train_test_split(test_size=0.1)
train_data = data_split["train"]
eval_data  = data_split["test"]

# === 1. 모델 로드 및 4bit 양자화 설정 ===
model_id = "google/gemma-3-4b-it"

if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype
)

# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    attn_implementation="eager",
    torch_dtype=torch_dtype
)

# 이미 학습된 LoRA 어댑터 불러오기 (.safetensors 기반 이어받기)
model = PeftModel.from_pretrained(base_model, "gemma3-4b-blog-qlora")

# === 2. 토크나이저 로드 ===
tokenizer = AutoTokenizer.from_pretrained(model_id)

# === 3. LoRA 설정 ===
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM"
)

# === 4. SFT 학습 설정 ===
train_args = SFTConfig(
    output_dir="gemma3-4b-blog-qlora",   # 같은 디렉토리에 덮어쓰기됨
    max_seq_length=512,
    packing=False,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    fp16=(torch_dtype == torch.float16),
    bf16=(torch_dtype == torch.bfloat16),
    logging_steps=10,
    save_strategy="epoch",
    push_to_hub=False,
    report_to="tensorboard"
)

# === 5. Trainer 생성 및 훈련 ===
trainer = SFTTrainer(
    model=model,
    args=train_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    tokenizer=tokenizer
)

trainer.train()

# === 6. 학습된 LoRA 어댑터 저장 ===
trainer.save_model()  # gemma3-4b-blog-qlora/ 에 adapter_model.safetensors 저장


# QLora 로딩
로딩, Lora 층을 model과 합체하는 코드

In [1]:
import os

# FlashAttention 및 SDPA 비활성화 (alignment 문제 방지용)
os.environ["FLASH_ATTENTION_FORCE_DISABLE"] = "1"
os.environ["SDPA_FORCE_DISABLE"] = "1"
os.environ["USE_FLASH_ATTENTION"] = "0"
os.environ["USE_MEM_EFFICIENT_ATTENTION"] = "0"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# 1. 기본(base) 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager"  # attention flash 미사용 !!!!!이거 안끄면 버그남 !!!!!!!!
)

# 2. 어댑터 적용
model = PeftModel.from_pretrained(base_model, "gemma3-4b-blog-qlora")  # 여기에 adapter_model.safetensors 있음

# 3. 토크나이저도 함께 로드
tokenizer = AutoTokenizer.from_pretrained("gemma3-4b-blog-qlora")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Qlora 추론코드
프롬프트와 추론하는 코드

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저는 이미 아래처럼 로딩된 상태라고 가정
# model = PeftModel.from_pretrained(base_model, "gemma3-4b-blog-qlora")
# tokenizer = AutoTokenizer.from_pretrained("gemma3-4b-blog-qlora")

prompt = "당신은 블로그를 포스팅하는 블로거입니다. 카페에 대한 글을 쓸 예정입니다.[가게이름:연남동 라이프카페] [맛:맛있음] [경치:주차장부터멋있음] [가격:비쌈] [분위기:넓음빈티지조용함] 의 내용으로 블로그를 포스팅해주세요. 이모지(👍💕..)나 특수기호($*#@)는 사용하지 마세요. 최대한 길게 쓰세요. 최대한 사람처럼 쓰세요."

# 토크나이즈
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 텍스트 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1000,         # 생성할 최대 토큰 수
        do_sample=True,             # sampling 방식 (True면 다양성 ↑)
        temperature=0.7,            # 샘플링 온도 (낮을수록 덜 창의적)
        top_p=0.9,                  # nucleus sampling
        repetition_penalty=1.1      # 반복 방지
    )

# 디코딩 및 출력
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


당신은 블로그를 포스팅하는 블로거입니다. 카페에 대한 글을 쓸 예정입니다.[가게이름:연남동 라이프카페] [맛:좀 짠데 맛있음] [경치:주차장부터멋있음] [가격:비쌈] [분위기:넓음빈티지조용함] 의 내용으로 블로그를 포스팅해주세요. 이모지(👍💕..)나 특수기호($*#@)는 사용하지 마세요. 최대한 길게 쓰세요. 최대한 사람처럼 쓰세요.[추가정보:용산구 연남고 2길 양제아파트 2층, 주차장부터 멋있고 경치가 서울 한복판이 보여 좋고, 고급진 닭갈비가 맛있었다고 해야함]
예쁜 카페 찾으러 다녀다니는 편이라서 이번에는 연남동에 있는 카페 중에서도 가장 높은 층에 위치해 있는 연남동 카페였는데 사진에서 보는 것보다 훨씬 더 넓었고 주차장이 있어서 주차가 편했고 야외석도 있어서 날씨 좋은 날에는 야외석에서 커피 한 잔 하는 것도 추천하고 싶어요 그리고 이곳의 특징인 건 바로 바깥쪽에 주차장을 만들었는데 그 주차장이 멋지고 특히 건물 뒤쪽으로 보이는 서울 한복판의 풍경이 정말 예뻤어요
한번은 용산구에서 연남동까지 가다가 양제아파트라고 써져있는 것을 보고 놀랐는데 그건 바로 연남동에 있는 카페였어요 그리고 카페 앞에도 작은 주차장이 있었는데 저는 처음에는 그냥 지나칠뻔 했어요 사실 카페 앞으로 주차장이 있는 곳은 제가 본 적 없어서 어떻게 이런 카페가 이곳에 있는지 궁금했어요 그리고 무엇보다도 카페 앞 주차장은 굉장히 멋진데요 용산에서 올라와서 올라오면 바로 보이는 풍경인데 카페 앞에 있는 주차장은 일반적인 주차장이 아니라 고급스러운 주차장이더라고요
연남동 카페는 카페 이름은 연남동 라이프카페이고 내부가 매우 넓고 자리도 많았어요 제가 방문했을 때는 평일이었는데 손님들이 많이 오지 않았고 그래서인지 모든 자리가 자유롭게 앉고 움직일 수 있는 공간이었어요
그리고 여기는 커피뿐만 아니라 다양한 음료를 판매하고 있었는데 저는 여기서 시그니처 메뉴인 아이스 카푸치노를 주문했어요 그리고 저는 매운 김치볶음밥과 닭갈비를 먹었는데 닭갈비가 고급스러워 보였다는 말로는 닭갈비 치고 가격이 

### end of token 설정 다시할것!! (코드 있는데 날라감. Fast api 사용할때 수정 사용)

# Qlora Auto Complete 기능
실험단계 todo : 기능개선

In [6]:
# 모델 및 토크나이저 로딩은 완료되었다고 가정합니다.

def generate_blog_post(cafe_details):
    """
    안정적인 블로그 포스트 생성을 위한 함수
    """
    # 1. 일관성을 위해 명확한 프롬프트 템플릿 사용
    prompt_template = "당신은 블로그를 포스팅하는 블로거입니다. {category}에 대한 글을 쓸 예정입니다.[가게이름:{cafe_name}] [맛:{taste}] [경치:{view}] [가격:{price}] [분위기:{atmosphere}] 의 내용으로 블로그를 포스팅해주세요. 이모지(👍💕..)나 특수기호($*#@)는 사용하지 마세요. 최대한 길게 쓰세요. 최대한 사람처럼 쓰세요."
    
    # 2. 템플릿에 맞춰 깨끗한 프롬프트 생성
    prompt = prompt_template.format(**cafe_details)

    # 항상 시작 토큰(bos_token)을 맨 앞에 추가
    if tokenizer.bos_token:
        prompt = tokenizer.bos_token + prompt

    # 깨끗하게 준비된 프롬프트를 토크나이징
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 종료 토큰 리스트 정의
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    # 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.65,  # 안정성을 위해 온도를 살짝 낮춤
            top_p=0.9,
            repetition_penalty=1.1
        )

    # 생성된 텍스트 디코딩 및 후처리
    # 입력 프롬프트 부분을 제외하고, <end_of_turn> 앞에서 잘라내기
    generated_text = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
    
    return generated_text.strip()

# 예시
gwangju_cafe_details = {
    "category" : "카페",
    "cafe_name": "Anuk .카페",
    "taste": "라떼가 달콤함",
    "view": "뷰",
    "price": "정보없음",
    "atmosphere": "아늑함"
}

# 함수를 호출하여 블로그 포스트 생성
blog_post = generate_blog_post(gwangju_cafe_details)
print("=== Generated Blog Post ===")
print(blog_post)

=== Generated Blog Post ===
안녕하세요
저는 매일 아침 6시반에 일어나는 버릇없는 사람입니다
그래서 저는 보통 오전 8시쯤에 일어나서 간단히 아침을 먹고
카페로 향합니다
오늘도 그런 날이었어요
아침 식사는 집에서 간단히 해결하고
카페로 향했습니다
그리고 바로 옆에 있는 카페 Anuk으로 갔습니다
이곳은 제가 가장 좋아하는 카페 중 하나인데요
그 이유는 무엇일까요
첫째는 이곳의 뷰가 너무 좋습니다
두 번째는 이곳의 커피 맛이 좋다는 것입니다
세 번째는 이곳의 분위기가 너무 좋아요
마지막으로 이곳의 디저트가 맛있다는 점입니다
Anuk은 신촌역 4번 출구에서 도보 5분 거리에 있습니다
카페 Anuk은 빨간 벽돌 건물 2층에 위치해 있어요
카페 Anuk은 주차 지원은 안되지만 대중교통이나 자전거로 오시는 분들은 편리하게 이용하실 수 있습니다
카페 Anuk은 음료와 디저트 메뉴가 다양해서
취향대로 골라 드실 수 있답니다
음료는 아이스커피 레몬티 초콜릿라떼 녹차티라미수케이크
디저트는 초코칩쿠키 바스크치즈케이크 시나모롤 등이 있어요
저는 오늘 날씨가 흐리고 비가 와서 그런지
초콜릿라떼가 더 맛있게 느껴졌는데요
초콜릿라떼는 기본 라떼에 초콜릿 시럽과 휘핑크림이 올라가는 메뉴예요
저는 초콜릿 시럽을 조금만 넣어달라고 했고
휘핑크림은 빼달라고 부탁드렸습니다
그럼에도 불구하고 휘핑크림이 올라가 있었는데
원래는 휘핑크림을 빼달라고 하지 않으셨어야 했는데
너무 아쉬웠어요
이런 건 주문할 때 미리 말씀드려야 하는 게 아닌가 싶네요
초콜릿 시럽은 정말 많이 넣으셔서
라떼가 단맛이 강했어요
다음에는 초콜릿 시럽을 조금만 넣어달라고 하거나
초콜릿 시럽을 넣지 않고 라떼만 주문해야겠어요
그리고 휘핑크림은 꼭 빼달라고 말씀드려야 합니다
초콜릿라떼가 너무 달아서
얼마 먹지 못하고 남겨버렸답니다
바스크치즈케이크는 정말 맛있었는데요
크림이 느끼하지 않고 바삭한 크러스트 위에 올려져 있어서
진짜 맛있었어요
제가 또 치즈케이크를 좋아하는데
여기 치즈케이크는 진짜 최고였어요
저는 매

In [20]:
def autocomplete(prompt: str, max_new_tokens: int = 64):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,          # 확정적 (greedy) 자동완성
            temperature=0.0,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id
        )

    # 전체 결과 디코딩
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 원래 prompt 이후 텍스트 추출
    completion = full_text[len(prompt):]

    # 줄바꿈(\n) 이전까지만 자르기
    return completion.split("\n")[0].strip()


In [14]:
autocomplete("2층 테라스 분위기가 ")


'훌륭한 카페에 다녀왔습니다.'